In [142]:
import spotipy, pandas as pd, json
from spotipy.oauth2 import SpotifyOAuth
from sklearn import preprocessing as pre

In [143]:
scope = 'playlist-modify-public playlist-modify-private'
red_url = 'http://localhost:8080'

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(redirect_uri=red_url,scope=scope))

In [144]:
songs = sp.playlist_tracks(playlist_id='https://open.spotify.com/playlist/37i9dQZEVXcD0rEV1cHpue?si=85e21e56fd774936', limit=100)

In [145]:
song_ids = []
i = 0
for song in songs['items']:
    song_ids.append(songs['items'][i]['track']['id'])
    i += 1

In [146]:
song_feats = sp.audio_features(tracks=song_ids)

In [147]:
aud_anal = []
aud_secs = []
j = 0
for song in song_ids:
    track = sp.audio_analysis(song_ids[j])['track']
    dur = track['duration']
    secs = len(sp.audio_analysis(song_ids[j])['sections'])
    aud_anal.append(track)
    aud_secs.append(60*secs/dur)
    j += 1

In [148]:
df_ids = pd.DataFrame(song_ids)
df_feats = pd.DataFrame(song_feats).drop(['speechiness', 'acousticness', 'instrumentalness', 'liveness', 'type', 'uri', 'track_href', 'analysis_url', 'duration_ms'], axis=1)
df_aud = pd.DataFrame(aud_anal)[['tempo_confidence','time_signature_confidence','key_confidence','mode_confidence']]
df_secs = pd.DataFrame({'secs/min': aud_secs})

In [149]:
df_tot = df_feats.join(df_secs)

In [150]:
df_tot.head()

,danceability,energy,key,loudness,mode,valence,tempo,id,time_signature,secs/min
0,0.644,0.671,11,-5.156,0,0.563,132.001,5vhrsx08dHh2QcVh1Rsiwt,4,2.632388
1,0.759,0.727,4,-6.030,1,0.284,124.045,3G9CA8tztFAwHCwE004dI3,4,1.999704
2,0.855,0.478,1,-8.310,1,0.536,153.074,2BSl2gSFHdzJAMus74qzNj,4,4.165841
3,0.854,0.601,1,-8.070,1,0.964,149.990,4FKzLOsMTwz9A00P9lVyrh,4,2.654672
4,0.675,0.593,0,-8.658,0,0.300,75.980,4pb9pQIU4jVYuf5qKsCi4s,4,3.941614


In [151]:
df_dummies = pd.get_dummies(df_tot[['key', 'time_signature']], columns=['key', 'time_signature'], drop_first=True)

In [152]:
to_norm = df_tot[['loudness', 'tempo', 'secs/min']].values
scaler = pre.MinMaxScaler()
normed = scaler.fit_transform(to_norm)
df_normed = pd.DataFrame(normed).rename(columns={0: 'loudness_normed', 1: 'tempo_normed', 2: 'secs/min_normed'})

In [153]:
df_tot.drop(['key', 'time_signature', 'loudness', 'tempo', 'secs/min'], axis=1, inplace=True)

In [154]:
df_tot = df_tot.join([df_dummies, df_normed]).set_index('id')

In [155]:
df_tot

,danceability,energy,mode,valence,key_1,key_2,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11,time_signature_4,loudness_normed,tempo_normed,secs/min_normed
id,,,,,,,,,,,,,,,,,,
5vhrsx08dHh2QcVh1Rsiwt,0.644,0.671,0,0.563,0,0,0,0,0,0,0,0,0,1,1,0.927633,0.493070,0.329420
3G9CA8tztFAwHCwE004dI3,0.759,0.727,1,0.284,0,0,1,0,0,0,0,0,0,0,1,0.885580,0.433786,0.052747
2BSl2gSFHdzJAMus74qzNj,0.855,0.478,1,0.536,1,0,0,0,0,0,0,0,0,0,1,0.775875,0.650095,1.000000
4FKzLOsMTwz9A00P9lVyrh,0.854,0.601,1,0.964,1,0,0,0,0,0,0,0,0,0,1,0.787422,0.627114,0.339164
4pb9pQIU4jVYuf5qKsCi4s,0.675,0.593,0,0.300,0,0,0,0,0,0,0,0,0,0,1,0.759130,0.075632,0.901945
59ClUavkkfRhryajZBn2Lf,0.495,0.759,1,0.503,0,0,0,0,0,0,1,0,0,0,1,0.735938,0.676570,0.227796
4NubkeFHcHFJ3Wy66VXMyC,0.917,0.531,0,0.535,0,0,0,0,0,0,0,0,1,0,1,0.771159,0.403779,0.433683
2lyVUYcK5jGsqqCUonK0n0,0.477,0.330,1,0.202,0,0,0,0,0,0,0,1,0,0,1,0.591926,1.000000,0.507905
1D3hpPheNncHHUPWpNyTYt,0.569,0.465,1,0.158,0,0,0,0,0,1,0,0,0,0,1,0.746091,0.627226,0.203198
